# 微博爬虫

## 1. 获取基础信息



## 2. 爬取粉丝列表

使用m.weibo.cn可以爬取大量粉丝，其api为    
<code> 
urlbase = 'https://m.weibo.cn/api/container/getIndex'  
param = {'containerid': '231051_-_fans_-_%d' % id,
             'type': 'all', 'since_id': since_id}
             </code>  
since_id 为粉丝展示页页码，从1开始

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import webbrowser


def get_followers_page(uid, since_id, save=0):
    url_base = 'https://m.weibo.cn/api/container/getIndex'
    param = {'containerid': '231051_-_fans_-_%d' % uid,
             'type': 'all', 'since_id': since_id}
    r = requests.get(url_base, param).json()
    if save != 0:
        f = open("%d_followers_%d.json" % (uid, since_id), 'w')
        json.dump(r, f)
        f.close()
    return r


def get_followers(uid):
    since_id = 0
    li = []
    while 1:
        since_id += 1
        r = get_followers_page(uid, since_id, 1)
        try:
            j = r['data']['cards'][0]['card_group']
        except:
            break
        print(since_id)
        for i in j:
            li.append(i['user']['id'])
    print(len(li))
    # print(r.url)
    return li

j = get_followers(6299499718)

## 3. 爬取关注人